In [1]:
cd ..

/Users/flamholz/Documents/workspace/ccm_evolution


In [2]:
import numpy as np
import pandas as pd
import re

from hnea_genes import *
from os import path

In [3]:
# A list of dictionaries specifying the data that we have
data_spec = [
    dict(path="data/Hnea/barseq/2017_11_03_miseq_barSeq1/",
         sets=[dict(key="setAS2 HCO", cond='5% CO2'), 
               dict(key="setAS3 LCO", cond='ambient CO2')],
         note="5% and ambient rep 1"),
    dict(path="data/Hnea/barseq/2017_11_03_miseq_barseq2/",
         sets=[dict(key="setAS2 HCO", cond='5% CO2'), 
               dict(key="setAS3 LCO", cond='ambient CO2')],
         note="5% and ambient rep 2"),
    dict(path="data/Hnea/barseq/2018_10_30_miseq_barSeq3",
         sets=[dict(key="setAS3 0-5", cond='0.5% CO2'), 
               dict(key="setAS5 10", cond='10% CO2')],
         note="0.5% and 10% CO2"),
    dict(path="data/Hnea/barseq/2018_10_30_miseq_barSeq4",
         sets=[dict(key="setAS4 0-5", cond='0.5% CO2'), 
               dict(key="setAS6 10", cond='10% CO2')],
         note="0.5% and 10% CO2"),
    dict(path="data/Hnea/barseq/2018_10_30_miseq_barSeq5",
         sets=[dict(key="setAS7 1-5", cond='1.5% CO2'), 
               dict(key="setAS8 5", cond='5% CO2')],
         note="1.5% and 5% CO2"),
    dict(path="data/Hnea/barseq/2018_10_30_miseq_barSeq6",
         sets=[dict(key="setAS10 1-5", cond='1.5% CO2'), 
               dict(key="setAS11 5", cond='5% CO2')],
         note="0.5% and 10% CO2"),
]

In [4]:
def load_single(spec, path):
    fit_df = pd.read_csv(path, sep='\t')
    cols = [s['key'] for s in spec['sets']]+['locusId']
    my_df = fit_df[cols].set_index('locusId')
    my_df.columns = [s['cond'] for s in spec['sets']]
    return my_df

def merge_dfs(df_list):
    con_df = pd.concat(df_list, axis=1, sort=True).reset_index()
    cols = list(con_df.columns)
    cols[0] = 'locus_id'
    con_df.columns = cols
    return con_df

def preprocess_barseq(specs):
    dfs_good = []
    dfs_all = []
    
    for my_spec in specs:
        # Merge the data called "good" by the barseq scripts, i.e. meets statistical quality filters.
        fpath = path.join(my_spec['path'], 'fit_logratios_good.tab')
        dfs_good.append(load_single(my_spec, fpath))
        
        # Also load the data with all the results for later inspection
        fpath = path.join(my_spec['path'], 'fit_logratios.tab')
        dfs_all.append(load_single(my_spec, fpath))
    
    con_df_good = merge_dfs(dfs_good)
    con_df_all = merge_dfs(dfs_all)
    
    return (con_df_good, con_df_all)

In [5]:
good_barseq_df, all_barseq_df = preprocess_barseq(data_spec)
good_barseq_df.head()

,locus_id,5% CO2,ambient CO2,5% CO2,ambient CO2,0.5% CO2,10% CO2,0.5% CO2,10% CO2,1.5% CO2,5% CO2,1.5% CO2,5% CO2
0,GFF1190,-0.000402,0.021627,-0.001932,0.076393,-0.062972,-0.044107,-0.097597,-0.114159,0.039218,0.022432,0.003704,-0.027252
1,GFF1209,0.222629,-0.117649,-0.130837,-0.047011,-0.087761,-0.134193,0.154560,-0.038279,0.168402,0.069838,0.227850,-0.037395
2,GFF1357,-0.612701,-0.344187,-0.504753,-0.315189,-1.174064,-0.906337,-1.203053,-0.942894,-1.133942,-1.033053,-1.031073,-1.220323
3,GFF1439,-0.277288,-0.268327,-0.234944,-0.290411,0.117596,-0.007212,0.229604,0.198248,-0.147151,-0.079903,-0.161343,-0.110442
4,GFF1496,0.676934,0.320859,0.515599,0.189416,-0.795918,-0.274587,-0.364849,-0.181126,0.111388,-0.022536,-0.053180,-0.028839


In [6]:
all_barseq_df.to_csv('data/Hnea/barseq/fit_logratios_all.csv', index=False)
good_barseq_df.to_csv('data/Hnea/barseq/fit_logratios_all_good.csv', index=False)

In [7]:
# Load the files again since loading them fixes duplicated column names using Pandas' logic.
all_barseq_df = pd.read_csv('data/Hnea/barseq/fit_logratios_all.csv')
good_barseq_df = pd.read_csv('data/Hnea/barseq/fit_logratios_all_good.csv')

In [8]:
# Make a long-form version of the above DataFrames for later plotting
def make_long_form(cond_df):
    # Make a long-form version of the barseq data as it's simpler for seaborn plotting.
    numeric_cols = (cond_df.dtypes == np.float64)
    long_df = cond_df.melt('locus_id', value_vars=cond_df.columns[numeric_cols])

    # Add categorical information.
    default_props = dict(kind='other', name=None)
    gene_tags = [genes_of_interest.get(row.locus_id, default_props)['kind'] 
                 for _, row in long_df.iterrows()]
    gene_names = [genes_of_interest.get(row.locus_id, default_props)['name'] 
                  for _, row in long_df.iterrows()]

    # Strip the .1 suffix in the condition name that comes from melt().
    conds = [re.sub('CO2.\d+', 'CO2', row.variable) for _, row in long_df.iterrows()]

    # Number the replicates for every condition
    pattern = re.compile(r'CO2.(\d+)')
    rep = [pattern.findall(row.variable) or ['0'] for _, row in long_df.iterrows()]
    rep = [int(i[0]) for i in rep]

    # Save everything in the DF.
    long_df['gene_tag'] = gene_tags
    long_df['gene_name'] = gene_names
    long_df['cond'] = conds
    long_df['rep'] = rep
    long_df['ccm_gene'] = long_df['gene_tag'] != 'other'
    return long_df

In [9]:
long_good_df = make_long_form(good_barseq_df)
long_good_df.to_csv('data/Hnea/barseq/fit_logratios_good_long.csv', index=False)

In [10]:
long_all_df = make_long_form(all_barseq_df)
long_all_df.to_csv('data/Hnea/barseq/fit_logratios_all_long.csv', index=False)

In [11]:
# Print out the number of mutants per gene in the library.
# CCM genes plotted in the main text figures
goif = genes_of_interest_filtered

strain_df = pd.read_csv('data/Hnea/barseq/2017_11_03_miseq_barSeq1/strain_fit.tab', sep='\t')
mask = strain_df['locusId'].isin(goif.keys())
mask = np.logical_and(mask, strain_df.used)
subset = strain_df[mask]
counts = subset.groupby('locusId').count()
counts['gene_name'] = [goif[k]['name'] for k in counts.index]
counts[['f', 'gene_name']]

,f,gene_name
locusId,,
HNEAP_RS01030,115,DAB2B
HNEAP_RS01035,59,DAB2A
HNEAP_RS04565,12,csos1D
HNEAP_RS04585,86,DAB1B
HNEAP_RS04595,70,DAB1A
HNEAP_RS04620,4,csos1B
HNEAP_RS04625,6,csos1A
HNEAP_RS04630,6,csos1C
HNEAP_RS04635,6,csos4B


In [12]:
# Average number of insertions used per CCM gene
counts['f'].mean()

39.857142857142854

In [21]:
# Generate starting material for SI table 1
genes_df = pd.read_csv('data/Hnea/barseq/2017_11_03_miseq_barSeq1/genes', sep='\t')
subset_df = genes_df[genes_df.locusId.isin(genes_of_interest.keys())].copy()
subset_df['gene_name'] = [genes_of_interest[k]['name'] for k in subset_df.locusId]

subset_df.sort_values('locusId').to_csv('/Users/flamholz/Desktop/tmp.csv')